In [47]:
from nltk.corpus import wordnet as wn
from nltk.tokenize import MWETokenizer

#get all multiword expressions
mwes = [x for x in wn.all_lemma_names() if '_' in x]

#foreach mwe in mwes generate a tuple
mwes = [tuple(x.split('_')) for x in mwes]
tokenizer = MWETokenizer(mwes, separator=' ')

text = "I live in New York and work in San Francisco and a cappella"
text = text.lower()
tokens = tokenizer.tokenize(text.split())
print(tokens)
mwes

['i', 'live in', 'new york', 'and', 'work in', 'san francisco', 'and', 'a cappella']


[('.22', 'caliber'),
 ('.22', 'calibre'),
 ('.38', 'caliber'),
 ('.38', 'calibre'),
 ('.45', 'caliber'),
 ('.45', 'calibre'),
 ('a', 'cappella'),
 ('a', 'couple', 'of'),
 ('a', 'few'),
 ('a', 'la', 'carte'),
 ('a', 'la', 'mode'),
 ('a', 'posteriori'),
 ('a', 'priori'),
 ('ad', 'hoc'),
 ('ad', 'hominem'),
 ('adequate', 'to'),
 ('agreed', 'upon'),
 ('air', 'sick'),
 ('al', 'dente'),
 ('all', 'important'),
 ('all', 'in'),
 ('all', 'over'),
 ('all', 'right'),
 ('anal', 'retentive'),
 ('ante', 'meridiem'),
 ('around', 'the', 'bend'),
 ('at', 'a', 'loss'),
 ('at', 'bay'),
 ('at', 'fault'),
 ('at', 'hand'),
 ('at', 'large'),
 ('at', 'loggerheads'),
 ('at', 'odds'),
 ('at', 'peace'),
 ('at', 'rest'),
 ('at', 'sea'),
 ('at', 'the', 'ready'),
 ('at', 'variance'),
 ('at', 'work'),
 ('attributive', 'genitive'),
 ('au', 'courant'),
 ('au', 'fait'),
 ('au', 'gratin'),
 ('au', 'jus'),
 ('au', 'naturel'),
 ('baking', 'hot'),
 ('below', 'the', 'belt'),
 ('bent', 'on'),
 ('better', 'off'),
 ('beyond', '

In [42]:
from nltk.tokenize import MWETokenizer

tokenizer = MWETokenizer([('New', 'York'), ('San', 'Francisco')], separator=' ')

text = "I live in New York and work in San Francisco"
tokens = tokenizer.tokenize(text.split())

print(tokens)

['I', 'live', 'in', 'New York', 'and', 'work', 'in', 'San Francisco']
